# Part 1: Set up

- Specify directories of source document & OpenAI API Key (linked to billig account)
- Import packages
- Specify ChatGPT Version

In [25]:
import_directory_pdf = "C:/Users/RLee/Downloads/scrape bdo uk.pdf"
# import_directory_txt = "C:/Users/RLee/Desktop/TAX BASE/bdo_uk_scrape.txt"

In [26]:
# ! pip install openai
# ! pip install os
# ! pip install python-dotenv

In [27]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Get openai_api_key using .env file
_ = load_dotenv(find_dotenv("C:/Users/RLee/Desktop/TAX BASE/openai_api_key.env")) # .env filepath
openai.api_key = os.environ["OPENAI_API_KEY"]

In [28]:
# Get recent LLM version
import datetime
current_date = datetime.datetime.now().date()
llm_name = "gpt-3.5-turbo-0301" if current_date < datetime.date(2023, 9, 2) else "gpt-3.5-turbo"

# Part 2: LLM Model Specification
- Import pdf & text processing

In [29]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# Import as pdf (with meta data elaboration)
loader = PyPDFLoader(import_directory_pdf)
doc = loader.load()

# split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
doc = text_splitter.split_documents(doc)

In [30]:
# with open(import_directory_txt) as f:
#     doc = f.read()

In [31]:
# Standard helper function
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# Part 3: Execution
- Specify a request/query **(User input)**
- Prompt engineering
- Calling LLM with prespecified architecture

In [32]:
query = "Pension tax?"

In [33]:
prompt = f""" 
Given a list of document summaries, your task is to assess each document strictly for relevance to the provided query. Exclude all documents that are 'likely irrelevant'—those that are only marginally related to the query.

Input:

Query: <{query}>
Document Summaries: <{doc}>

Procedure:

1. Essence Extraction: Discern the core essence and essential points of the query.
2. Relevance Assessment: Review the titles and descriptions of each document summary. Determine whether the document is likely to be relevant or maybe relevant to the query's key points. Disregard any document that does not appear to closely align with the query's essence.

Output:

For each document that is determined to be relevant or maybe relevant, present the following details:

- Conclusion: [Relevant/Maybe Relevant]
- Key Objective of the Query: [Concise summary of the query's key points]
- Document Details:
  - Title: [Title of the document]
  - Date: [Publication date of the document]
  - Description: [Overview of the document's main themes and points]
  - Link: [Direct URL to the full document]
- Reasoning: [Justification for the relevance assessment, connecting the document's title and description to the query]
- Relevant Snippet: [A specific part of the document summary that exemplifies its relevance]

Documents deemed 'likely irrelevant' should not be included in the output. Focus only on presenting those with a clear connection or potential connection to the query's essence.
"""


In [34]:
print("Question:\n", query, "\n\n=============================================")

# Track total runtime
from datetime import datetime
_start_time = datetime.now()

response = get_completion(prompt)
print(response)

    
# Print execution time
print(f"\n\n\n===============================\nTotal runtime:  {datetime.now() - _start_time}")


Question:
 Pension tax? 



InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 12405 tokens. Please reduce the length of the messages.